In [1]:
import pandas as pd 
import pyodbc
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

print("Libraries Loaded")

Libraries Loaded


In [31]:
# Download the VADER lexicon for sentiment analysis
nltk.download('vader_lexicon') 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...


True

In [13]:
# Define a function to fetch data from a SQL database 
conn_str = "Driver={SQL Server};Server=DESKTOP-8EG0K3J\SQLEXPRESS;Database=PortfolioProject_MarketingAnalytics;Trusted_Connection=yes;"

In [20]:
# Establishing a connection to the database
conn = pyodbc.connect(conn_str)

query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"

In [ ]:
df = pd.read_sql(query, conn)

conn.close()

C:\Users\Acer\AppData\Local\Temp\ipykernel_4880\73150797.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [22]:
df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [36]:
customer_reviews_df = df

In [37]:
sia = SentimentIntensityAnalyzer()
sia

In [38]:
def calculate_sentiment(customer_reviews_df):
    sentiment = sia.polarity_scores(customer_reviews_df)
    return sentiment['compound'] 

In [40]:
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else: return 'Mixed Positive'
    else: 
        if rating >= 4:
            return 'Positive'
        elif rating <= 2: 
            return 'Negative'
        else: return 'Neautral'

In [39]:
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score <0.0:
        return '-0.49 to 0.0'
    else: return '-1.0 to -0.5'

In [41]:
# Applying sentiment analysis to calculate sentiment scores for each review
customer_reviews_df['sentiment_score'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)
customer_reviews_df['sentiment_category'] = customer_reviews_df.apply( lambda row: categorize_sentiment(row['sentiment_score'], row['Rating']), axis= 1)

In [44]:
customer_reviews_df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,sentiment_score,sentiment_category
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative


In [43]:
customer_reviews_df.to_csv('Customer_reviews_sentiment_analysis.csv', index = False)